<a href="https://colab.research.google.com/github/SushovitNanda/SemEval-Food-Hazards/blob/main/PC_Albert_66.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import necessary libraries
%%capture
!pip install transformers
!pip install datasets --upgrade
#!pip install torch
import os
import pandas as pd
!pip install evaluate
import evaluate
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, EarlyStoppingCallback
from datasets import Dataset
import warnings
warnings.filterwarnings("ignore")

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!wget https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_train.csv
!wget https://raw.githubusercontent.com/SushovitNanda/SemEval-Food-Hazards/main/Datasets/incidents_val.csv

In [ ]:
# Load the dataset
train = pd.read_csv('incidents_train.csv')

# Combine 'title' and 'text' columns to create input data
train['input_text'] = train['title'] + " " + train['text']

# Encode the target labels
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['product-category'])

# Train-test split using stratification
train_df, test_df = train_test_split(
    train[['input_text', 'label']],
    test_size=0.2,
    stratify=train['label'],
    random_state=42
)

# Convert the dataframes into Hugging Face's Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load the ALBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

# Define a tokenization function
def tokenize_function(example):
    return tokenizer(example['input_text'], truncation=True)

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set up a data collator to pad inputs dynamically
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load ALBERT model for sequence classification
num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained(
    "albert-base-v2",
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

# Load the F1 metric and specify macro averaging
f1_metric = evaluate.load("f1")

# Define the compute_metrics function to maximize F1 macro average
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Compute the F1 macro average
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    return {"f1": f1["f1"]}

# Update training arguments to focus on F1 score
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Use F1 score as the metric for saving best model
    greater_is_better=True       # Ensure higher F1 is considered better
)

# Redefine the Trainer with the updated compute_metrics and early stopping callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # Use F1 macro for evaluation
)

# Add EarlyStoppingCallback to the Trainer with patience of 4 epochs
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4))

In [6]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,1.317100,1.389121,0.363707
2,1.082900,1.072417,0.456099
3,0.609400,0.990594,0.506945
4,0.439200,1.023439,0.500048
5,0.361400,0.946413,0.572157
6,0.139900,0.995698,0.602600
7,0.312600,1.114721,0.644261
8,0.241400,1.299137,0.663787
9,0.090400,1.360107,0.649971
10,0.009700,1.374309,0.663617


TrainOutput(global_step=6108, training_loss=0.477272845471989, metrics={'train_runtime': 5294.616, 'train_samples_per_second': 11.516, 'train_steps_per_second': 1.442, 'total_flos': 1167222216264720.0, 'train_loss': 0.477272845471989, 'epoch': 12.0})

In [7]:
# Evaluate the model
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

# Classification report
target_names = label_encoder.classes_
print(classification_report(test_df['label'], preds, target_names=target_names))


                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.86      1.00      0.92        12
                      cereals and bakery products       0.75      0.83      0.79       134
     cocoa and cocoa preparations, coffee and tea       0.70      0.71      0.71        42
                                    confectionery       0.64      0.47      0.54        34
dietetic foods, food supplements, fortified foods       0.73      0.73      0.73        26
                                    fats and oils       1.00      0.75      0.86         4
                                   feed materials       0.00      0.00      0.00         1
                   food additives and flavourings       0.00      0.00      0.00         2
                           food contact materials       0.00      0.00      0.00         1
                            fruits and vegetables       0.88      0.79      0.83       10